# Land Scraper – a Parcel Estimator Tool

## Purpose
The Parcel Estimator tool returns the following parcel data based on zip code:
- Average price per acre.
- Average price.
- average acreage.
- Most popular seller.
- listing popularity for the area.


## How to Use
Replace the variable "zip" with your given zip code and run all lines of code. The csv and charts exported will be found in the output folder.


## Tool Functionality
1. Import: Import previous tool data. 
2. Scrape: Scrape RE sites for parcel data into dataframe.
3. Clean: Clean dataframe's new rows.
4. Visualize: Plot charts, identifying the differences in key metrics.
5. Export: Export dataframe as CSV and charts as PNG files.


## Resources
- Dataset: import previous csv's to compare.
- Libraries: Pandas, Numpy, BeautifulSoup, Matplotlib, Plotly

## 1. Setup

In [1]:
# Import general libraries
import pandas as pd
import numpy as np
from datetime import date

# import libraries necessary for web scraping
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

# Import data cleaning and enriching file
from src.cleaning import*

# This code ignores warnings from Pandas not to overwrite data
pd.options.mode.chained_assignment = None

In [2]:
# import and check csv file: list of all US zip codes
all_us_zips = pd.read_csv('input/geo-data.csv')
all_us_zips.sample(5)

,state_fips,state,state_abbr,zipcode,county,city
21318,37,North carolina,NC,28723,Jackson,Cullowhee
29991,51,Virginia,VA,22485,King George,King george
17772,33,New hampshire,NH,03273,Merrimack,South sutton
15374,28,Mississippi,MS,39269,Hinds,Jackson
8806,19,Iowa,IA,50042,Audubon,Brayton


## 2. Scrape the web for parcel data

### Information to save (9 criteria): 
- Zip Code
- Location
- State
- Country
- parcel size (acres)
- parcel price
- listing name
- listing author
- posting date

In [106]:
reg = "(.+)([A-Z]{2})\s.+(\b(\d{5})\b)"
reg2 = '(.+)([A-Z]{2})\s(\d{5}).+[A-Z]?'
test = "123 Cornudas Alabama, TX 79847, USA"
a = re.match(reg2, test)
print(a.groups())

('123 Cornudas Alabama, ', 'TX', '79847')


In [116]:
def scrape_page(url):
    """ SEARCH FOR ALL LISTING RESULTS ON A PAGE """
    html = requests.get(url)
    soup = BeautifulSoup(html.content, "html.parser")
    
    # Return location data of all results on current page to new lists.
    location_raw = soup.select('div.post-location-snippet.bmargin.font-sm')
    
    # Clean location data.
    location_list = [i.get_text().strip() for i in location_raw]
    print(location_list)
    
    # All regex criteria
    pre_re_state = "(.+)\b[A-Z][A-Z]\b.+" # regex criteria for everything before state code
    pre_re_zip   = '(.+)\b\d{5}\b.+'      # regex criteria for everything before zip code
    re_state     = '.+\b([A-Z][A-Z])\b.+'     # regex criteria for state code: 2 capital letters surrounded by spaces
    re_zip       = '.+\b(\d{5})\b.+'          # regex criteria for zip code: 5 digits surrounded by spaces
    re_usa       = '.+\bUSA\b.+'
    test = "(Cornudas), TX 79847, USA"

    # All location lists to pass into dict into df
    addresses = []
    zips = []
    states = []
    
    # Search all raw location data on a page for info to pass into location lists
    for raw_address in location_list:
        if re.findall(re_state, raw_address): # regex search for state code. Store address, state code, zip code
            addresses.append((re.findall(pre_re_state, raw_address)[0].strip))
            states.append(re.search(re_state))
            zips.append(re.search(re_zip))
        elif re.search(re_zip, raw_address): # regex search for zip code. Store address, NA state code, zip code
            addresses.append(re.search(pre_re_zip).strip)
            states.append(np.nan) # fill state code w NaN
            zips.append(re.search(re_zip))
        elif re.search(re_usa, raw_address): # if address is only USA then fill address, state code, zip code w NaN
            addresses.append(np.nan)
            states.append(np.nan)
            zips.append(np.nan)
        else: # if only address is available, fill that and NaN values for the rest
            print(pre_re_state, raw_address)
            try:
                addresses.append(re.match('(.+)([A-Z]{2})\s(\d{5}).+[A-Z]?', raw_address).groups()[0]) #TEST
                states.append(re.match('(.+)([A-Z]{2})\s(\d{5}).+[A-Z]?', raw_address).groups()[1]) #TEST
                zips.append(re.match('(.+)([A-Z]{2})\s(\d{5}).+[A-Z]?', raw_address).groups()[2]) #TEST
                #addresses.append(raw_address)
            except:
                pass
            
            
    # Return all parcel size, price, name data to new list.
    parcel_size = soup.select('div.post-location-snippet.tmargin span.inline-block')
    parcel_price = soup.select('div.bg-secondary.hpad.font-lg.text-center.line-height-xl')
    listing_name = soup.select('a.h3.bold.bmargin.center-block')

    # clean parcel data.
    size = [i.get_text().strip() for i in parcel_size]
    size = [i.lstrip("Acres:\n") for i in size]
    price = [i.get_text().lstrip('\n$').replace(",","") for i in parcel_price]
    listing = [i.get_text().strip() for i in listing_name]

    
    
    # Return author and date data of all results on page 1 to new lists.
    author_and_date = soup.select('div.col-xs-8.col-sm-10.nolpad.font-sm.bmargin.posted_meta_data')

    # Clean author and date data.
    auth_and_date = [author_and_date[i].get_text().strip() for i in range(len(author_and_date))] # send to cleaning function later

    # Split author and date data into separate lists.
    # Every string has the same format, ie: 'Posted\n04/14/2022 \n\nby\n\nLandJakes'
    date = []
    author = []
    for elem in auth_and_date:
        date.append(elem.replace("\n", "")[6:].replace("by", "").split(" ")[0]) 
        author.append(elem.replace("\n", "")[6:].replace("by", "").split(" ")[1])

        
    print('addresses', len(addresses), 'zips', len(zips), 'states', len(states), 'size', len(size), 'price', len(price), 'listing', len(listing), 'author', len(author), 'date', len(date))    
    while len(addresses) != len(price):
        addresses.append(np.nan)
        states.append(np.nan)
        zips.append(np.nan)
            
    # Add all lists to a new (temp) dataframe.
    dict_to_merge = {'Address':addresses, 'Zip Code':zips, 'State':states,
                    'Parcel Size (acres)':size, 'Parcel Price':price, 'Listing Name':listing, 
                    'Listing Author':author, 'Post Date':date}
    new_search_df = pd.DataFrame(dict_to_merge)
    
    return new_search_df

### Input zip code and loop through all pages.

In [118]:
zip_list = [79847, 93535] # temp list
df = pd.DataFrame()
for zip_code in zip_list: #all_us_zips['zipcode']: # Loop through zip code dataframe, calling search for each zip
    
    # Set up the url to scrape
    pg = 1 # set page value to 1
    url = f'https://www.landmodo.com/properties?page={pg}&q={zip_code}&property_status=Land+for+Sale'
    
    # Loop through pages
    while empty_page(url) is False:
        df = pd.concat([df, scrape_page(url)], axis=0)
        pg +=1
        url = f'https://www.landmodo.com/properties?page={pg}&q={zip_code}&property_status=Land+for+Sale'
df

['Cornudas, TX 79847, USA', 'Cornudas, TX 79847, USA', 'Cornudas, TX 79847, USA', 'Cornudas, TX 79847, USA', 'Cornudas, TX 79847, USA']
(.+[A-Z][A-Z.+ Cornudas, TX 79847, USA
(.+[A-Z][A-Z.+ Cornudas, TX 79847, USA
(.+[A-Z][A-Z.+ Cornudas, TX 79847, USA
(.+[A-Z][A-Z.+ Cornudas, TX 79847, USA
(.+[A-Z][A-Z.+ Cornudas, TX 79847, USA
addresses 5 zips 5 states 5 size 5 price 5 listing 5 author 5 date 5
['P7XQ+93G Lancaster, CA 93535', 'P78X+QJ Lancaster, CA 93535, USA', 'Lancaster, CA 93535', 'Lancaster, CA 93535', 'Lancaster, CA 93535, USA', 'Lancaster, CA 93535', 'M4RC+JRF Lancaster, CA 93535, USA', 'Lancaster, CA 93535, USA', 'Lancaster, CA 93535, USA', 'M3VW+JCJ Lancaster, CA 93535, USA', 'Lancaster, CA 93535, USA', 'Lancaster, CA 93535, USA']
(.+[A-Z][A-Z.+ P7XQ+93G Lancaster, CA 93535
(.+[A-Z][A-Z.+ P78X+QJ Lancaster, CA 93535, USA
(.+[A-Z][A-Z.+ Lancaster, CA 93535
(.+[A-Z][A-Z.+ Lancaster, CA 93535
(.+[A-Z][A-Z.+ Lancaster, CA 93535, USA
(.+[A-Z][A-Z.+ Lancaster, CA 93535
(.+[A-Z][A-

,Address,Zip Code,State,Parcel Size (acres),Parcel Price,Listing Name,Listing Author,Post Date
0,"Cornudas,",79847,TX,20.18,18000.00,LAND FOR SALE – How to grow Old & Rich togethe...,LandJakes,04/14/2022
1,"Cornudas,",79847,TX,20.00,18000.00,Invest 20 Acres or Neighboring Lots in Cornuda...,LandJakes,03/04/2022
2,"Cornudas,",79847,TX,20.00,18000.00,"Invest 20 Acres Land in Hudspeth County, TX. N...",LandJakes,03/04/2022
3,"Cornudas,",79847,TX,20.00,18000.00,Invest 20 Acres or Neighboring Lots in Cornuda...,LandJakes,02/10/2022
4,"Cornudas,",79847,TX,18.00,18000.00,"Invest 20 Acres Land in Hudspeth County, TX. N...",LandJakes,02/10/2022
0,"P78X+QJ Lancaster,",93535,CA,1.25,5995.00,"Your Wait Is Over, Rush Today For Sumer Sale!!!",Patrick,08/10/2022
1,"Lancaster,",93535,CA,2.50,7995.00,"2.50 Acres Lancaster, Los Angeles County, CA -...",Patrick,07/29/2022
2,"M4RC+JRF Lancaster,",93535,CA,4.80,11995.00,GOOD DEAL IN CA,Patrick,07/29/2022
3,"Lancaster,",93535,CA,19.24,29995.00,19.24 Acres Vacant Land in Lancaster!,Patrick,07/15/2022
4,"Lancaster,",93535,CA,2.50,6995.00,"A Worth Having Country Retreat in Lancaster, CA!",Patrick,07/11/2022


## 3. Cleaning
See "cleaning.py" in the "src" folder. All cleaning is done inside of a separate cleaning file.

### Additional cleaning:
- Data with incorrect zip codes removed.
- Searches with zero hits warn the user of such.
- Missing values are replaced with null.

## 4. Visualize

Important information to chart:
- Average price, acreage per zip
- Price, Acreage range per zip
- Zip code heat map (by hits, by price per acre)
- previous charts, by state or sub-state county

### See "Visualization.ipynb" for continuation.

## 5. Export dataframe for use in Visualization file.

In [5]:
# dfnew.to_csv(f'output/landmodo_search_{date.today()}.csv', index="False")